## Spinner with Threads

Idea: 
Below, we have 2 functions (`slow` and `spin`).  
Start a function (`slow`) that blocks for 3 seconds while animating characters in the terminal to let the user know that the program is “thinking” and not stalled.  
The `spin` function makes an animated spinner displaying each character in the string `\|/-` in the same screen position. When the slow computation finishes, the line with the spinner is cleared and the result is shown (42).

In [1]:
import itertools
import time
from threading import Thread, Event


def spin(msg: str, done: Event) -> None:  # This function will run in a separate thread. Event is used to synchronize threads 
    for char in itertools.cycle(r'\|/-'):  # This is an infinite loop because itertools.cycle yields one character at a time
        status = f'\r{char} {msg}'  # Trick for text-mode animation: move the cursor back to the start of the line with the carriage return ASCII control character ('\r').
        print(status, end='', flush=True)
        if done.wait(.1):  # .wait returns True when the event is set() by another thread; if the timeout elapses, it returns False
            break  # Exit the infinite loop
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')  # <6>


def slow() -> int:
    time.sleep(3)  # slow() will be called by the main thread. Imagine this is a slow API call over the network.
    return 42

&#128212; `time.sleep()` blocks the calling thread but releases the GIL, allowing other Python threads to run.

Now let's call the above 2 functions

In [2]:
def supervisor() -> int:  # supervisor will return the result of slow
    done = Event()  # Event is used to coordinate the main thread and spinner thread
    spinner = Thread(target=spin, args=('thinking!', done))  # Run a new thread with the spin function as target
    print(f'spinner object: {spinner}')  # the spinner object will be <Thread(Thread-1, initial)>, where initial means the thread is not started yet
    spinner.start()  # Start the spinner thread.
    result = slow()  # Call slow, which blocks the main thread. Meanwhile, the spinner thread is running.
    done.set()  # Set the Event flag to True; this will terminate the for loop inside the spin function
    spinner.join()  # Wait until the spinner thread finishes.
    return result


def main() -> None:
    result = supervisor()  # Run the supervisor function.
    print(f'Answer: {result}')


main()

spinner object: <Thread(Thread-4 (spin), initial)>
Answer: 42  
